In [1]:
import numpy as np

In [2]:
traindata=np.loadtxt('train.txt', delimiter=',')
testdata=np.loadtxt('test.txt', delimiter=',')
traindata

array([[ 50.,  21.,  77., ...,  48.,  22.,   2.],
       [ 53.,   8.,  77., ...,  48.,  24.,   4.],
       [ 49.,   0.,  79., ...,  32.,   2.,   1.],
       ...,
       [ 56.,   0.,  95., ...,  39.,   0.,   1.],
       [ 37.,  -1.,  76., ...,  37.,   0.,   1.],
       [ 40.,  -3., 100., ...,  62.,   2.,   1.]])

In [3]:
traindata=np.mat(traindata)
traindata.shape

(4513, 10)

In [4]:
X=traindata[:, :-1]
d=X.shape[1]

In [5]:
n=len(X)
n

4513

In [6]:
O=traindata[:, 9]
O

matrix([[2.],
        [4.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]])

In [7]:
from sklearn.preprocessing import OneHotEncoder

In [8]:
onehot_encoder = OneHotEncoder(sparse=False)
O = O.reshape(len(O), 1)
onehot_O = onehot_encoder.fit_transform(O)
onehot_O=np.mat(onehot_O) #onehot coding for y
onehot_O.shape

/Users/fuxia/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


(4513, 7)

In [9]:
p=7
bo=np.random.uniform(-0.1, 0.1,size=(p,1))
bo

array([[0.04134831],
       [0.08814553],
       [0.00294339],
       [0.00606539],
       [0.03348939],
       [0.02078923],
       [0.04116802]])

In [10]:
m=15
bh=np.random.uniform(-0.1, 0.1,size=(m,1))
bh

array([[ 0.05204996],
       [-0.00552775],
       [-0.03238816],
       [-0.06119533],
       [ 0.05161961],
       [-0.08583721],
       [ 0.04086407],
       [-0.0296422 ],
       [ 0.05649959],
       [ 0.03838487],
       [-0.06328422],
       [-0.01999992],
       [ 0.08809283],
       [ 0.00428506],
       [-0.0682771 ]])

In [11]:
wh=np.random.uniform(-0.1, 0.1,size=(d, m))
wh.shape#initial weight for input and hidden layer

(9, 15)

In [12]:
wo=np.random.uniform(-0.1, 0.1,size=(m, p))

In [13]:
def ReLU_activation(net):
    for i in range(m):
        if net[i]<=0:
            net[i]=0           
    return net  

In [14]:
def Softmax(net):
    denomiter=np.sum(list(map(lambda x:np.exp(x), net)))
    for i in range(p):
        net[i]=np.exp(net[i])/denomiter
    return net

In [15]:
def feed_forward_z(X_i, wh, bh):
    z_i=ReLU_activation(bh+np.dot(wh.T, X_i.T))
    return z_i

In [16]:
def feed_forward_o(z_i, wo, bo):
    o_i=Softmax(bo+np.dot(wo.T, z_i))
    return o_i

In [17]:
z=feed_forward_z(X[1], wh, bh)
o=feed_forward_o(z, wo, bo)

In [18]:
X[1].shape

(1, 9)

In [19]:
z

matrix([[0.        ],
        [3.25023724],
        [0.        ],
        [4.79527124],
        [0.        ],
        [1.78156035],
        [4.50038997],
        [0.        ],
        [0.73670624],
        [1.84438899],
        [0.        ],
        [0.50943948],
        [5.85587833],
        [0.        ],
        [0.        ]])

In [20]:
o

matrix([[0.19063509],
        [0.08663105],
        [0.20195645],
        [0.17164386],
        [0.11032073],
        [0.13933837],
        [0.09947445]])

In [21]:
def back_phase_new(o, z, y_i, wo):
    
    allones_z=np.ones(shape=(m,1))

    So=o-y_i.T
    
    for i in range(m): 
        if z[i]<=0:
            allones_z[i]=0
    Sh=np.multiply(allones_z, np.dot(wo, So))
    return So, Sh

In [22]:
So, Sh=back_phase_new(o, z, onehot_O[1], wo)

In [23]:
onehot_O[1].shape

(1, 7)

In [24]:
So.shape

(7, 1)

In [25]:
Sh.shape

(15, 1)

In [26]:
def gradient_b(bo, bh, lr, so, sh):
    bo=bo-lr*so
    bh=bh-lr*sh
    return bo, bh

In [27]:
def gradient_w(wo, wh, so, sh, lr, z, o, x):
    wo=wo-lr*np.dot(z, so.T)
    wh=wh-lr*np.dot(x.T, sh.T)
    return wo, wh

In [28]:
#gradient_w(wo, wh, So, Sh, lr, z, o, X[1])

In [29]:
import random
def model(wo, wh, bo, bh, lr, X, y, maxiter):
    order_x=list(range(X.shape[0]))
    t=0
    while True:
        random.shuffle(order_x)
        for i in order_x:
            #forward
            z_i=feed_forward_z(X[i], wh, bh)
            o_i=feed_forward_o(z_i, wo, bo)
            #backward
            So_i, Sh_i=back_phase_new(o_i, z_i, y[i], wo)
            #update bo, bh
            bo, bh=gradient_b(bo, bh, lr, So_i, Sh_i)
            #update wo, wh
            wo, wh=gradient_w(wo, wh, So_i, Sh_i, lr, z_i, o_i, X[i])
        t=t+1
        if t >= maxiter:
            break
    return bo, bh, wo, wh

In [30]:
lr=0.00001
maxiter=10
Bo, Bh, Wo, Wh=model(wo, wh, bo, bh, lr, X, onehot_O, maxiter)
Bo

/Users/fuxia/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


matrix([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]])

In [31]:
def pred_y(bh, bo, wh, wo, x):
    o=np.zeros(p*len(x)).reshape(len(x),p)
    for i in range(len(x)):
        z_i=feed_forward_z(x[i], wh, bh)
        o[i]=feed_forward_o(z_i, wo, bo).T
    return o

In [32]:
predict_y=pred_y(Bh, Bo, Wh, Wo, X)
predict_y.shape

(4513, 7)

In [33]:
def adjust_yi(a, x):
    y=np.zeros(shape=(len(x), p))
    for k in range(len(x)):
        for i in range(p):
            for j in range(p):
                if a[k][i]==np.max(a[k]):
                    y[k][i]=1
    return y

In [34]:
pred_y_train=adjust_yi(predict_y, X)
pred_y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [35]:
def accuracy(y_pred,y_true):
    total_num = y_true.shape[0]
    count_good = 0
    for i in range(len(y_true)):
        if (y_pred[i] ==y_true[i]).sum()==7:
            count_good +=1
    return count_good/total_num

In [36]:
np.sum(pred_y_train, axis=0)

array([0., 0., 0., 0., 0., 0., 0.])

In [37]:
np.sum(onehot_O, axis=0)

matrix([[3432.,   37.,  132.,  662.,  233.,    6.,   11.]])

In [38]:
accuracy(pred_y_train, onehot_O )

0.0

For test data

In [39]:
testdata=np.mat(testdata)
X_test=testdata[:, :-1]

In [40]:
testdata

matrix([[ 55.,   0.,  81., ...,  88.,  64.,   4.],
        [ 56.,   0.,  96., ...,  44.,   4.,   4.],
        [ 50.,  -1.,  89., ...,  40.,   2.,   1.],
        ...,
        [ 55.,   0.,  77., ...,  65.,  42.,   4.],
        [ 37.,   0., 103., ...,  85.,  20.,   1.],
        [ 56.,   2.,  98., ...,  46.,   4.,   4.]])

In [41]:
X_test.shape

(14500, 9)

In [42]:
Output=testdata[:, 9]
Output

matrix([[4.],
        [4.],
        [1.],
        ...,
        [4.],
        [1.],
        [4.]])

In [43]:
onehot_encoder = OneHotEncoder(sparse=False)
Output = Output.reshape(len(Output), 1)
Y_test = onehot_encoder.fit_transform(Output)
Y_test=np.mat(Y_test) #onehot coding for y
Y_test.shape

/Users/fuxia/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


(14500, 7)

In [44]:
predict_test=pred_y(Bh, Bo, Wh, Wo, X_test)

In [45]:
predict_test.shape

(14500, 7)

In [46]:
adjust_y_test=adjust_yi(predict_test, X_test)
adjust_y_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [47]:
np.sum(adjust_y_test, axis=0)

array([0., 0., 0., 0., 0., 0., 0.])

In [48]:
np.sum(Y_test, axis=0)

matrix([[1.1478e+04, 1.3000e+01, 3.9000e+01, 2.1550e+03, 8.0900e+02,
         4.0000e+00, 2.0000e+00]])

In [49]:
accuracy(adjust_y_test, Y_test)

0.0